In [7]:
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

w, h = model_wh('0x0')
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path('cmu'), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path('cmu'), target_size=(w, h))

image = common.read_imgfile('./images/p2.jpg', None, None)
image_h, image_w = image.shape[:2]
t = time.time()
humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
elapsed = time.time() - t
log = []
for human in humans:
    centers = {}
    for i in range(common.CocoPart.Background.value):
        if i not in human.body_parts.keys():
            continue
        body_part = human.body_parts[i]
        center = (int(body_part.x * image_w + 0.5), int(body_part.y * image_h + 0.5))
        centers[i] = center
    log.append(centers)

print(len(humans))
img = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[2019-10-02 11:49:06,010] [TfPoseEstimator] [INFO] loading graph from /home/yinghuit/Documents/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)
I1002 11:49:06.010591 139631239681856 estimator.py:310] loading graph from /home/yinghuit/Documents/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)
W1002 11:49:06.013800 139631239681856 deprecation_wrapper.py:119] From /home/yinghuit/Documents/tf-pose-estimation/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W1002 11:49:06.015469 139631239681856 deprecation_wrapper.py:119] From /home/yinghuit/Documents/tf-pose-estimation/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

W1002 11:49:09.031294 139631239681856 deprecation_wrapper.py:119] From /home/yinghuit/Documents/tf-pose-estimation/tf_pose/estimator.py:330: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_gr

2


## To Draw Human Pose Centers and Lines

In [9]:
# Testing

image_h = 1080
image_w = 1920
jsonstring = '[{0: (908, 40), 1: (892, 62), 2: (860, 58), 5: (928, 60), 6: (976, 68), 7: (988, 70), 16: (886, 38), 14: (900, 36), 15: (910, 36)}, {8: (530, 504), 1: (526, 470), 2: (534, 470), 11: (518, 506), 5: (514, 472)}, {17: (584, 274), 5: (606, 308), 6: (600, 348), 7: (588, 356)}]'

In [11]:
import cv2
import numpy as np

from tf_pose import common
from tf_pose.common import CocoPart

img = np.zeros([image_h,image_w,3],dtype=np.uint8)
img.fill(255)
# log = eval(jsonstring)
print(len(log))
for obj in log:
    for k, v in obj.items():
        cv2.circle(img, v, 3, common.CocoColors[k], thickness=3, lineType=8, shift=0)
    for pair_order, pair in enumerate(common.CocoPairsRender):
        if pair[0] not in obj.keys() or pair[1] not in obj.keys():
            continue
        cv2.line(img, obj[pair[0]], obj[pair[1]], common.CocoColors[pair_order], 3)
cv2.imshow('draw', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

3
